In [ ]:
pip install rasterio

     |████████████████████████████████| 18.1MB 247kB/s 


In [ ]:
import os
import numpy as np
import imageio
import imageio as im
import gdal
import rasterio 
from glob import glob
import matplotlib.pyplot as plt

### IMPORTANDO VARIÁVEIS DO METADADO PARA CALIBRAÇÃO RADIOMÉTRICA###

In [ ]:
## 1 CONSTANTES PARA CALIBRAÇÃO RADIOMÉTRICA (transformando de ND para Radiância TOA em Wm-2sr-1um-1)

## 1.1 Extraindo fator de escala multiplicativo da banda a partir do metadata da cena (MULT)


MULT = 'MULT_BAND_10'
def encontrar(MULT):
    with open('caminho_metadata.txt', 'r') as MTL:
        linhas = MTL.readlines()
        for i in range(len(linhas)):
            if 'MULT_BAND_10' in linhas[i]:
                a = linhas[i].split('=')[-1].strip('\n')
                
    return a
ML = encontrar(MULT)

################################################################################################################################

## 1.2 Extraindo fator de escala aditivo da banda a partir do metadata da cena (ADD)


ADD = 'ADD_BAND_10'
def encontrar(ADD):
    with open('caminho_metadata.txt', 'r') as MTL:
        linhas = MTL.readlines()
        for i in range(len(linhas)):
            if 'ADD_BAND_10' in linhas[i]:
                b = linhas[i].split('=')[-1].strip('\n')
                
    return b
AL = encontrar(ADD)

### IMPORTANDO BANDA THERMAL EM ND ###

In [ ]:
#IMPORTANDO IMAGEM INTEIRA
banda10 = glob('caminho_da_banda.TIF')  #inserir o diretório no qual a banda se encontra
with rasterio.open(banda10[0]) as src:
    b10= src.read(1)
b_10 = b10.astype(float)
kwargs = src.meta.copy()
kwargs.update({'dtype': 'float64'})

#PARA RECORTAR IMAGEM:
#window = rasterio.windows.Window(2720, 3132, 1, 1) #col_off, row_off, width, height  #exemplo para 1 pixel
#window
#banda10 = glob('/content/LC08_L1TP_220081_20180314_20180320_01_T1_B10.TIF') #inserir diretório da imagem
#with rasterio.open(banda10[0]) as src:
    #b10= src.read(1, window=window)
#b_10 = b10.astype(float)
#kwargs = src.meta.copy()
#kwargs.update({'dtype': 'float64'})

### EXECUTANDO A CALIBRAÇÃO RADIOMÉTRICA ###

In [ ]:
ml = float(ML) #fator de escala multiplicativo da banda
al = float(AL) #fator de escala aditivo da banda

#função para calibração radiométrica

def radcal(b_10):
    for i in b_10:
        radiance = (ml*b_10) + al
    return radiance

Lsens = radcal(b_10).astype(float) #obtendo Radiância TOA em Wm-2sr-1um-1

with rasterio.open('Radiance_TOA_BX.tiff', 'w', **kwargs) as dst: #exportando imagem final com calibração radiométrica 
    dst.write(Lsens.astype(rasterio.float64),1)

### RECUPERAÇÃO TEMPERATURA DA SUPERFÍCIE TERRESTRE (TST) PELA INVERSÃO DA EQUAÇÃO DA TRANSFERÊNCIA RADIATIVA/RADIATIVE TRANSFER EQUATION (RTE) ###

In [ ]:
#DEFINIÇAO DAS VARIÁVEIS PARA RECUPERAR TST PELA INVERSÃO DA RTE

#Definição do comprimento de onda efetivo da banda

lamb = 10.904 #Aqui, no exemplo, é da banda 10, sensor TIRS

#Definição emissividade do alvo no comprimento de onda efetivo

emis = 0.9798 #Exemplo de emissividade do quartzo calculada a partir de radiância registrada em campo com uFTIR

#Definição das constantes derivadas da equação de Planck

c1= 1.19104*(10**8) #Wµm4m-2sr-1 
c2= 14387.7 #µmK

#Definição das variáveis atmosféricas (podem ser obtidas pela calculadora de parâmetros de correção atmosférica (BARSI et al., 2003,2005)

Lu = 2.60 #Radiância atmosférica no sentido ascendente ou upwelling radiance
Ld = 4.12 #Radiância atmosférica no sentido descendente ou downwelling radiance
trans = 0.7 #transmitância atmosférica


### ESTIMANDO A TST PELA INVERSÃO DA RTE ###

In [ ]:
#Função para estimar a TST pela RTE

def temp_RTE(Lsens):
    for i in Lsens:
        a = c1/((lamb**5)*(((Lsens-Lu)/(emis*trans))-((1-emis)/emis)*Ld))
        b = (np.log(a+1))**-1
        temp = (c2/lamb)*b
    return temp

temp_RTE = temp_RTE(Lsens).astype(float) #obtendo TST em Kelvin

with rasterio.open('Land_Surface_Temperature.tiff', 'w', **kwargs) as dst: #exportando imagem final de recuperação da TST para o diretório do projeto
    dst.write(temp_RTE.astype(rasterio.float64),1)